In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel("C:/Users/Arshad/Desktop/Python/Stocks_Capital_Gains_Report.xlsx")

# Find the headers
header = df.iloc[2,0]

# Platform and brokerage charges
charges=df.iloc[7:19,0:2]
charges_colname = ['Groww charges','Amount']
charges.columns = charges_colname
charges['Amount'] = charges['Amount'].astype(float)
charges.reset_index(drop=True,inplace=True)

# 3.Merge total and dividends as total gain
toth=df.index[df['Name']=='Realised P&L']
total=df.iloc[toth[0]+1:toth[0]+4,0:2]
total_colname = ['Category','Amount Gained']
total.columns = total_colname
total.reset_index(drop=True,inplace=True)
total['Amount Gained'] = total['Amount Gained'].astype(float)

others=df.iloc[toth[0]+9:toth[0]+11,0:2]
others_colname = ['Category','Amount Gained']
others.columns = others_colname
others.reset_index(drop=True,inplace=True)
others['Amount Gained'] = others['Amount Gained'].astype(float)

totgain = pd.concat([total,others],axis=0,ignore_index=True)
totgain.loc['5',:] = ['Total Charges (STT, DP, Brokerage)',charges['Amount'].sum()]
total_gain = totgain.loc[totgain['Category'] != 'Total Charges (STT, DP, Brokerage)', 'Amount Gained'].sum()
total_charges = totgain.loc[totgain['Category'] == 'Total Charges (STT, DP, Brokerage)', 'Amount Gained'].sum()
totgain.loc['6',:] = ['Net Profit/Loss',total_gain - total_charges]

# 4.Stock differentiation
shth=df.index[df['Name']=='Intraday trades']
stocks=df.iloc[shth[0]+2:,:]
empty = stocks[stocks.isnull().all(axis=1)]
emtrow=empty.index
stock_colname = ['Stock','ISIN','Quantity','Buy date','Buy price','Buy value','Sell date','Sell price','Sell value','P&L','Remarks']

# 5.Intraday stocks
intra=df.iloc[shth[0]+2:emtrow[0],:]
intra.columns = stock_colname
intra.reset_index(drop=True,inplace=True)
intra.loc[:,'Quantity'] = pd.to_numeric(intra['Quantity'], errors='coerce')
intra.loc[:,'Buy value'] = pd.to_numeric(intra['Buy value'], errors='coerce')
intra.loc[:,'Buy price']  = pd.to_numeric(intra['Buy price'], errors='coerce')
intra.loc[:,'Sell value'] = pd.to_numeric(intra['Sell value'], errors='coerce')
intra.loc[:,'Sell price'] = pd.to_numeric(intra['Sell price'], errors='coerce')
intra.loc[:,'P&L'] = pd.to_numeric(intra['P&L'], errors='coerce')

# 6.Short term stocks
shth=df.index[df['Name']=='Short Term trades']
short=df.iloc[shth[0]+2:emtrow[1],:]
short.columns = stock_colname
short.reset_index(drop=True,inplace=True)
short.loc[:,'Quantity'] = pd.to_numeric(short['Quantity'], errors='coerce')
short.loc[:,'Buy value'] = pd.to_numeric(short['Buy value'], errors='coerce')
short.loc[:,'Buy price']  = pd.to_numeric(short['Buy price'], errors='coerce')
short.loc[:,'Sell value'] = pd.to_numeric(short['Sell value'], errors='coerce')
short.loc[:,'Sell price'] = pd.to_numeric(short['Sell price'], errors='coerce')
short.loc[:,'P&L'] = pd.to_numeric(short['P&L'], errors='coerce')

# 7.Long term stocks
lngh=df.index[df['Name']=='Long Term trades']
lng=df.iloc[lngh[0]+2:,:]
empty_helper = lng[lng.isnull().all(axis=1)]
empty_h = empty_helper.index
long=df.iloc[lngh[0]+2:empty_h[0],:]
long.columns = stock_colname
long.reset_index(drop=True,inplace=True)
long.loc[:,'Quantity'] = pd.to_numeric(long['Quantity'], errors='coerce')
long.loc[:,'Buy value'] = pd.to_numeric(long['Buy value'], errors='coerce')
long.loc[:,'Buy price']  = pd.to_numeric(long['Buy price'], errors='coerce')
long.loc[:,'Sell value'] = pd.to_numeric(long['Sell value'], errors='coerce')
long.loc[:,'Sell price'] = pd.to_numeric(long['Sell price'], errors='coerce')
long.loc[:,'P&L'] = pd.to_numeric(long['P&L'], errors='coerce')

# Analysis on Intraday
intra_summary = intra.groupby("Stock").agg({'Quantity': 'sum','Buy value': 'sum','Sell value': 'sum','P&L': 'sum'}).reset_index()
intra_summary.sort_values(by='P&L', ascending=False, inplace=True)
intra_summary['Quantity'] = pd.to_numeric(intra_summary['Quantity'], errors='coerce')
intra_summary['Buy value'] = pd.to_numeric(intra_summary['Buy value'], errors='coerce')
intra_summary['Sell value'] = pd.to_numeric(intra_summary['Sell value'], errors='coerce')
intra_summary['P&L'] = pd.to_numeric(intra_summary['P&L'], errors='coerce')

# Analysis on Short term
short_summary = short.groupby("Stock").agg({'Quantity': 'sum','Buy value': 'sum','Sell value': 'sum','P&L': 'sum'}).reset_index()
short_summary.sort_values(by='P&L', ascending=False, inplace=True)
short_summary['Quantity'] = pd.to_numeric(short_summary['Quantity'], errors='coerce')
short_summary['Buy value'] = pd.to_numeric(short_summary['Buy value'], errors='coerce')
short_summary['Sell value'] = pd.to_numeric(short_summary['Sell value'], errors='coerce')
short_summary['P&L'] = pd.to_numeric(short_summary['P&L'], errors='coerce')

# Analysis on Long term
long_summary = long.groupby("Stock").agg({'Quantity': 'sum','Buy value': 'sum','Sell value': 'sum','P&L': 'sum'}).reset_index()
long_summary.sort_values(by='P&L', ascending=False, inplace=True)
long_summary['Quantity'] = pd.to_numeric(long_summary['Quantity'], errors='coerce')
long_summary['Buy value'] = pd.to_numeric(long_summary['Buy value'], errors='coerce')
long_summary['Sell value'] = pd.to_numeric(long_summary['Sell value'], errors='coerce')
long_summary['P&L'] = pd.to_numeric(long_summary['P&L'], errors='coerce')

# Displaying dataframes together
def display_side_by_side(*args, titles=cycle([''])):
    html_str = ''
    for df, title in zip(args, chain(titles, cycle(['</br>']))):
        html_str += '<th style="text-align:center"><td style="vertical-align:top">'
        html_str += f'<h2 style="text-align: center;">{title}</h2>'
        html_str += df.to_html().replace('table', 'table style="display:inline"')
        html_str += '</td></th>'
    display_html(html_str, raw=True)
display_side_by_side(totgain, intra_summary, short_summary,long_summary, titles=['Overall Summary','Intraday Summary', 'Short Term Stock Summary', 'Long Term Stock Summary']) 

Overall Summary 
 
 
 
 Category 
 Amount Gained 
 
 
 
 
 0 
 Intraday P&L 
 1662.510 
 
 
 1 
 Short Term P&L 
 16141.030 
 
 
 2 
 Long Term P&L 
 -12290.450 
 
 
 3 
 Dividends 
 5548.125 
 
 
 4 
 Buyback P&L 
 0.000 
 
 
 5 
 Total Charges (STT, DP, Brokerage) 
 5114.142 
 
 
 6 
 Net Profit/Loss 
 5947.073 
 
 
 Intraday Summary 
 
 
 
 Stock 
 Quantity 
 Buy value 
 Sell value 
 P&L 
 
 
 
 
 11 
 Minda Industries Ltd 
 160 
 56157.67 
 55216.21 
 11307.82 
 
 
 1 
 Avantel Ltd 
 152 
 27654.68 
 47601.62 
 6492.27 
 
 
 10 
 Mahanagar Gas Ltd 
 329 
 63088.96 
 46485.14 
 6117.32 
 
 
 14 
 Tata Chemicals Ltd 
 158 
 20129.05 
 12485.69 
 5358.99 
 
 
 13 
 Sangam India Ltd 
 178 
 31041.63 
 44319.90 
 4773.18 
 
 
 2 
 Bharat Electronics Ltd 
 298 
 16511.03 
 32225.47 
 2938.33 
 
 
 4 
 Gretex Corporate Services Ltd 
 123 
 37776.35 
 40428.42 
 2663.28 
 
 
 8 
 Ksolves India Ltd 
 43 
 30567.24 
 6064.69 
 1754.99 
 
 
 0 
 Anand Rathi Wealth Management Ltd 
 155 
 57683.98 
 26323.75 
 1389.77 
 
 
 12 
 NIIT Technologies Ltd 
 183 
 42264.05 
 39692.13 
 -497.01 
 
 
 3 
 Easy Trip Planners Ltd 
 35 
 18072.53 
 9995.96 
 -873.82 
 
 
 9 
 Lloyds Metals Ltd 
 24 
 13036.76 
 13896.17 
 -3477.85 
 
 
 5 
 Gujarat Themis Biosyn Ltd 
 290 
 36095.31 
 43929.97 
 -4505.51 
 
 
 6 
 Jeena Sikho Ltd 
 145 
 32057.78 
 35706.76 
 -7226.58 
 
 
 7 
 Key Corp Leasing And Investments Ltd 
 98 
 19444.18 
 25387.03 
 -9552.67 
 
 
 Short Term Stock Summary 
 
 
 
 Stock 
 Quantity 
 Buy value 
 Sell value 
 P&L 
 
 
 
 
 1 
 HDFC BANK 
 283 
 50703.26 
 57758.49 
 11417.08 
 
 
 8 
 TCS 
 85 
 46697.46 
 52279.45 
 9184.29 
 
 
 2 
 ICICI BANK 
 172 
 33272.64 
 9306.34 
 5253.64 
 
 
 5 
 L&T 
 372 
 59659.51 
 63442.73 
 2173.28 
 
 
 6 
 RELIANCE 
 136 
 19918.32 
 35600.58 
 1271.70 
 
 
 0 
 BHARTI AIRTEL 
 203 
 28789.46 
 26322.71 
 425.52 
 
 
 9 
 WIPRO 
 83 
 22930.49 
 34510.23 
 -919.85 
 
 
 4 
 ITC 
 26 
 10447.72 
 10667.47 
 -2315.25 
 
 
 7 
 TATA STEEL 
 171 
 26180.55 
 27788.18 
 -4393.74 
 
 
 3 
 INFOSYS 
 100 
 12251.67 
 33760.66 
 -5955.64 
 
 
 Long Term Stock Summary 
 
 
 
 Stock 
 Quantity 
 Buy value 
 Sell value 
 P&L 
 
 
 
 
 13 
 Tata Consultancy Services Ltd 
 110 
 16694.83 
 26181.90 
 7578.34 
 
 
 3 
 HDFC Bank Ltd 
 66 
 18180.77 
 33861.54 
 7458.46 
 
 
 14 
 Tata Motors Ltd 
 283 
 16562.41 
 47864.59 
 1628.41 
 
 
 8 
 Larsen & Toubro Ltd 
 60 
 10729.77 
 8136.08 
 1601.97 
 
 
 2 
 Bharti Airtel Ltd 
 108 
 32230.40 
 54390.48 
 1583.69 
 
 
 9 
 Mahindra & Mahindra Ltd 
 81 
 19463.86 
 17207.18 
 908.33 
 
 
 6 
 Infosys Ltd 
 79 
 18110.36 
 23028.04 
 832.80 
 
 
 0 
 Axis Bank Ltd 
 236 
 45130.12 
 34942.26 
 -685.01 
 
 
 5 
 ICICI Bank Ltd 
 272 
 44609.89 
 51708.55 
 -895.24 
 
 
 10 
 Reliance Industries Ltd 
 121 
 12398.24 
 18953.27 
 -3111.13 
 
 
 7 
 Kotak Mahindra Bank Ltd 
 107 
 3475.58 
 25916.59 
 -3213.20 
 
 
 12 
 Sun Pharmaceutical Industries Ltd 
 78 
 13013.81 
 16392.15 
 -4215.44 
 
 
 11 
 State Bank of India 
 107 
 27819.89 
 27580.43 
 -6915.20 
 
 
 4 
 Hindustan Unilever Ltd 
 125 
 39034.61 
 47837.73 
 -7081.06 
 
 
 1 
 Bajaj Finance Ltd 
 191 
 68903.98 
 56098.60 
 -7766.17